# Loss Functions in DidgeLab

This notebook explains the **modular loss API** for evolutionary optimization of didgeridoo shapes. You combine **loss components** (e.g. frequency tuning, scale tuning, peak count) into a **CompositeTairuaLoss**. The evolution algorithm minimizes the total loss.

**Contents:**
1. CompositeTairuaLoss – how to combine components
2. Frequency tuning – align drone/toots to target notes
3. Scale tuning – pull resonances toward a musical scale
4. Peak quantity & amplitude – encourage many, strong resonances
5. Harmonic & timbre components – integer harmonics, Q-factor, shimmer, etc.

## Setup

Add the project root so we can import `didgelab`. Then import the loss classes and NumPy.

In [ ]:
import sys
sys.path.insert(0, "../../../src")

import numpy as np
from didgelab import (
    CompositeTairuaLoss,
    FrequencyTuningLoss,
    ScaleTuningLoss,
    PeakQuantityLoss,
    PeakAmplitudeLoss,
    QFactorLoss,
    ModalDensityLoss,
    IntegerHarmonicLoss,
    NearIntegerLoss,
    StretchedOddLoss,
    HighInharmonicLoss,
    HarmonicSplittingLoss,
    note_to_freq,
)

# CompositeTairuaLoss

**CompositeTairuaLoss** runs the acoustical simulation for a shape, detects impedance peaks, and evaluates each registered **loss component** on those peaks. The **total loss** is the sum of all component losses.

**Total loss (scalar for evolution):**

$$L_{total} = \sum_{k} L_k$$

where $L_k$ are the individual component losses (e.g. $L_{freq}$, $L_{scale}$). The evolution uses only $L_{total}$; the per-component values are stored in the loss dict for analysis.

**Usage:**
- Build a composite loss with `CompositeTairuaLoss(max_error=5.0)` (max_error controls simulation frequency resolution).
- Add components with `add_component(name, component)`.
- Use `.loss(shape)` in evolution; it returns a dict with keys for each component and `"total"`.
- Set `loss.target_freqs` (array of target frequencies in Hz) if you use `init_standard_evolution` for plotting.

In [ ]:
# Example: build a loss with two components
target_freqs_hz = np.array([73.4, 146.8])  # e.g. D1, D2
target_freqs_log = np.log2(target_freqs_hz)
target_impedances = np.array([-1.0, -1.0])  # -1 = ignore impedance; use [0,1] to target amplitudes

loss = CompositeTairuaLoss(max_error=5.0)
loss.add_component("freq", FrequencyTuningLoss(target_freqs_log, target_impedances, weights=[1.0, 1.0]))
loss.add_component("volume", PeakAmplitudeLoss(target_min_amplitude=0.25, weight=20.0))
loss.target_freqs = target_freqs_hz  # optional, for init_standard_evolution

# When used in evolution: result = loss.loss(shape)  -> {"freq": ..., "volume": ..., "total": ...}

---
## Notebooks in this folder

| Notebook | Description |
|----------|-------------|
| **00_overview** | This notebook: intro, setup, CompositeTairuaLoss. |
| **01_frequency_tuning** | Frequency Tuning Loss – align peaks to target frequencies. |
| **02_scale_tuning** | Scale Tuning Loss – pull resonances toward a musical scale. |
| **03_peak_quantity** | Peak Quantity Loss – encourage many resonances. |
| **04_peak_amplitude** | Peak Amplitude Loss – reward strong resonances. |
| **05_q_factor** | Q-Factor Loss – resonance sharpness. |
| **06_modal_density** | Modal Density (Shimmer) Loss – clustering for beating. |
| **07_integer_harmonic** | Integer Harmonic Loss – pure harmonic series. |
| **08_near_integer** | Near-Integer (Stretched) Loss – piano-like stretch. |
| **09_stretched_odd** | Stretched Odd Harmonics – hollow/woody timbre. |
| **10_high_inharmonic** | High Inharmonic Loss – metallic/dissonant timbre. |
| **11_harmonic_splitting** | Harmonic Splitting Loss – gritty doublets. |
| **12_full_example** | Full evolution run with composite loss. |
